# 📚 CacheBackedEmbeddings - 임베딩 캐싱으로 성능 최적화하기

## 📖 목차
1. [CacheBackedEmbeddings란?](#1-cachebackedembeddings란)
2. [LocalFileStore를 사용한 영구 캐싱](#2-localfilestore를-사용한-영구-캐싱)
3. [InMemoryByteStore를 사용한 임시 캐싱](#3-inmemorybytestore를-사용한-임시-캐싱)

---

## 📋 개요

**CacheBackedEmbeddings**는 임베딩 계산 결과를 저장해두고 재사용할 수 있게 해주는 **똑똑한 캐싱 시스템**입니다! 💡

### 🤔 왜 캐싱이 필요할까요?

임베딩 생성은 **시간과 비용이 많이 드는 작업**입니다:
- **⏱️ 시간 소모**: 같은 텍스트를 반복해서 임베딩으로 변환
- **💰 비용 발생**: OpenAI API 호출할 때마다 요금 부과
- **🔄 중복 작업**: 이미 처리한 텍스트를 다시 처리하는 비효율

### 🎯 CacheBackedEmbeddings의 핵심 기능

**CacheBackedEmbeddings**는 마치 **사전(辭典)**처럼 작동합니다:

1. **📝 첫 번째 요청**: "사과"라는 단어의 임베딩을 계산하고 저장
2. **⚡ 두 번째 요청**: "사과"를 다시 만나면 저장된 결과를 즉시 반환
3. **🔍 해시 기반**: 텍스트를 해시값으로 변환하여 키로 사용

### 🏗️ 주요 구성 요소

#### `from_bytes_store` 메소드의 핵심 매개변수:

- **`underlying_embeddings`** 🤖: 실제 임베딩을 생성하는 모델 (예: OpenAIEmbeddings)
- **`document_embedding_cache`** 💾: 임베딩을 저장할 저장소 (ByteStore)
- **`namespace`** 🏷️: 캐시 충돌을 방지하기 위한 구분자

### ⚠️ 중요한 주의사항

**namespace 매개변수**를 반드시 설정하세요! 다른 임베딩 모델을 사용할 때 **캐시 충돌**을 방지할 수 있습니다.

**예시**: `text-embedding-3-small`과 `text-embedding-ada-002`를 구분하여 저장

In [ ]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv(override=True)

In [ ]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("LangChain-Tutorial")

## 2. LocalFileStore를 사용한 영구 캐싱 💾

**LocalFileStore**는 임베딩 결과를 **컴퓨터 하드디스크에 파일로 저장**하는 방식입니다.

### 🏪 실생활 비유: 동네 편의점 창고

동네 편의점을 생각해보세요:
- **📦 창고**: LocalFileStore (컴퓨터의 폴더)
- **🏷️ 상품 바코드**: 텍스트 해시값 (임베딩을 찾는 키)
- **📋 재고 목록**: 저장된 임베딩 데이터

### ✅ LocalFileStore의 장점
- **🔒 영구 보관**: 프로그램을 종료해도 캐시가 남아있음
- **💰 비용 절약**: 한 번 계산한 임베딩은 계속 재사용
- **⚡ 빠른 속도**: 두 번째부터는 즉시 로드

### ⚠️ LocalFileStore의 단점  
- **💾 저장 공간**: 하드디스크 용량을 차지
- **🧹 관리 필요**: 오래된 캐시 파일 정리 필요

이제 실제로 로컬 파일 시스템을 사용하여 임베딩을 저장하고 **FAISS 벡터 스토어**를 사용하여 검색하는 예제를 살펴보겠습니다.

In [ ]:
from langchain.storage import LocalFileStore
from langchain_openai import OpenAIEmbeddings
from langchain.embeddings import CacheBackedEmbeddings
from langchain_community.vectorstores.faiss import FAISS

# OpenAI 임베딩을 사용하여 기본 임베딩 설정
embedding = OpenAIEmbeddings(model="text-embedding-3-small")

# 로컬 파일 저장소 설정 - "./cache/" 폴더에 캐시 파일 저장
store = LocalFileStore("./cache/")

# 캐시를 지원하는 임베딩 생성
cached_embedder = CacheBackedEmbeddings.from_bytes_store(
    underlying_embeddings=embedding,  # 실제 임베딩을 수행할 모델
    document_embedding_cache=store,   # 캐시를 저장할 저장소
    namespace=embedding.model,        # 모델별로 캐시를 구분하기 위한 네임스페이스
)

In [ ]:
# 현재 캐시 저장소에 저장된 키들을 확인 (아직 아무것도 없음)
list(store.yield_keys())

### 📄 문서 처리 과정

이제 **문서를 로드하고 처리하는 전체 과정**을 단계별로 진행해보겠습니다:

1. **📖 문서 로드**: 텍스트 파일에서 내용 읽기
2. **✂️ 청크 분할**: 긴 텍스트를 작은 단위로 나누기  
3. **🔢 임베딩 생성**: 각 청크를 벡터로 변환 (캐싱 적용!)
4. **🗃️ 벡터 저장소**: FAISS를 사용해 검색 가능한 형태로 저장

In [ ]:
from langchain.document_loaders import TextLoader
from langchain_text_splitters import CharacterTextSplitter

# 텍스트 파일에서 문서 로드
raw_documents = TextLoader("./data/appendix-keywords.txt").load()

# 문자 단위로 텍스트 분할 설정 (1000자씩 나누고, 겹치는 부분은 0자)
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)

# 로드된 문서를 설정된 크기로 분할
documents = text_splitter.split_documents(raw_documents)

In [ ]:
# 첫 번째 실행 - 모든 문서를 임베딩하고 FAISS 데이터베이스 생성 (시간 측정)
%time db = FAISS.from_documents(documents, cached_embedder)

### ⚡ 캐시 효과 확인하기

이제 **캐싱의 진짜 위력**을 확인해볼 시간입니다! 

같은 문서로 벡터 저장소를 다시 생성하면, **이미 계산된 임베딩을 캐시에서 가져와서** 훨씬 더 빠르게 처리됩니다.

### 🏃‍♂️ 속도 비교 실험
- **첫 번째 실행**: OpenAI API 호출 + 임베딩 계산 + 캐시 저장
- **두 번째 실행**: 캐시에서 바로 로드 ⚡

결과적으로 **몇 초에서 몇 밀리초로** 처리 시간이 대폭 단축됩니다!

In [ ]:
# 두 번째 실행 - 캐싱된 임베딩을 사용하여 훨씬 빠르게 FAISS 데이터베이스 생성
%time db2 = FAISS.from_documents(documents, cached_embedder)

## 3. InMemoryByteStore를 사용한 임시 캐싱 🧠

**InMemoryByteStore**는 **컴퓨터 메모리(RAM)에만 임시로 저장**하는 방식입니다.

### 🧠 실생활 비유: 사람의 단기 기억

사람의 기억을 생각해보세요:
- **📝 단기 기억**: InMemoryByteStore (프로그램 실행 중에만 유지)
- **📚 장기 기억**: LocalFileStore (파일로 영구 보관)
- **😴 잠들면 잊어버림**: 프로그램 종료 시 모든 캐시 사라짐

### ✅ InMemoryByteStore의 장점
- **🚀 초고속**: 메모리 접근이 파일 접근보다 빠름
- **🧹 자동 정리**: 프로그램 종료 시 자동으로 정리됨  
- **💾 공간 절약**: 하드디스크 용량을 사용하지 않음

### ⚠️ InMemoryByteStore의 단점
- **⏰ 일시적**: 프로그램을 재시작하면 캐시가 모두 사라짐
- **🧠 메모리 사용**: RAM을 많이 사용할 수 있음

### 🎯 언제 사용할까?

- **임시 작업**: 한 번의 실행 중에만 캐시가 필요한 경우
- **테스팅**: 개발 중 빠른 테스트를 위해
- **메모리 풍부**: RAM이 충분하고 디스크 사용을 피하고 싶을 때

아래에서는 **비영구적인 InMemoryByteStore**를 사용하여 동일한 캐시된 임베딩 객체를 생성하는 예시를 보여줍니다.

In [ ]:
from langchain.embeddings import CacheBackedEmbeddings
from langchain.storage import InMemoryByteStore

# 메모리 내 바이트 저장소 생성 (프로그램 종료시 사라짐)
store = InMemoryByteStore()

# 캐시 지원 임베딩 생성 (메모리 기반)
cached_embedder = CacheBackedEmbeddings.from_bytes_store(
    embedding,                # 기본 임베딩 모델
    store,                    # 메모리 내 저장소
    namespace=embedding.model # 네임스페이스로 캐시 구분
)